In [1]:
import cvxopt
import numpy as np
from svmutil import *
from numpy import linalg
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd 
import numpy.linalg as la
MIN_SUPPORT_VECTOR_MULTIPLIER = 1e-4
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
class SVM(object):
    def __init__(self, kernel, c=1):
        self._kernel = kernel
        self._c=float(c)
    
    def get_alphas(self,x,y):
        n_samples, n_features = x.shape
        K = self._gram_matrix(x)
        P = matrix(np.outer(y, y) * K)
        q = matrix(-1 * np.ones(n_samples))
        G_std = matrix(np.diag(np.ones(n_samples) * -1))
        h_std = matrix(np.zeros(n_samples))
        G_slack = matrix(np.diag(np.ones(n_samples)))
        h_slack = matrix(np.ones(n_samples) * self._c)
        G = matrix(np.vstack((G_std, G_slack)))
        h = matrix(np.vstack((h_std, h_slack)))
        A = matrix(y, (1, n_samples))
        b = matrix(0.0)
        solvers.options['show_progress'] = False
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        alphas = np.array(solution['x'])
        return alphas
    
    def _gram_matrix(self, x):
        n_samples, n_features = x.shape
        K = np.zeros((n_samples, n_samples))
        for i, x_i in enumerate(x):
            for j, x_j in enumerate(x):
                K[i,j] = self._kernel(x_i, x_j)
        return K
    
    def get_w_b(self,x,y,alphas):
        K = self._gram_matrix(x)
        sv_i= alphas > MIN_SUPPORT_VECTOR_MULTIPLIER
        self.al = []
        self.sv_x = []
        self.sv_y = []
        sv_index=[]
        for i in range(len(sv_i)):
            if bool(sv_i[i]) is True :
                sv_index.append(i)
                self.al.append(a[i])
                self.sv_x.append(x[i])
                self.sv_y.append(y[i])
        self.b = 0.0
        self.al = np.array(self.al)
        self.sv_x = np.array(self.sv_x)
        self.sv_y = np.array(self.sv_y)
        sv_index = np.array(sv_index).reshape([-1,1])
        for n in range(len(self.al)):
            self.b += self.sv_y[n]
            for m in range(len(self.al)):
                self.b -= self.al[m]*self.sv_y[m]*K[sv_index[n],sv_index[m]]
            #self.b -= np.sum(self.al * self.sv_y * K[sv_index[n],sv_i])
        self.b /= len(self.al)
        if self._kernel == Kernel.linear:
            self.w = np.zeros(x.shape[1])
            for n in range(len(self.al)):
                self.w += self.al[n] * self.sv_y[n] * self.sv_x[n]
        else:
            self.w = None
    
    def predictor(self,x):
        pre=[]
        if self.w is not None:
            pre = np.dot(x, self.w) + self.b
        else:
            pre = np.zeros(len(x))
            for i in range(len(x)):
                s = 0
                for a, sv_y, sv_x in zip(self.al, self.sv_y, self.sv_x):
                    s += a * sv_y * self._kernel(x[i], sv_x)
                pre[i] = s
            pre=pre + self.b
        return np.sign(pre)
    
    def accuracy(self,y, y_pred):
        y_pred = np.reshape(y_pred,[-1,1])
        return np.sum(y == y_pred) / len(y)

In [3]:
class Kernel(object):
    def linear():
        return lambda x, y: np.inner(x, y)
    
    def gaussian(sigma=5):
        return  lambda x, y: np.exp(-np.sqrt(la.norm(x-y) ** 2 / (2 * sigma ** 2)))

In [4]:
data =pd.read_csv('ex8a.csv')
data=data.dropna(how='all')
data=np.array(data)
split=StratifiedShuffleSplit(n_splits=1,test_size=0.25,random_state=42)
for train_index,val_index in split.split(data,data[:,0]):
    train_set = data[train_index]
    val_set = data[val_index]

y_train = np.reshape(np.array(train_set[:,0]),[-1,1])
X_train = np.array(train_set[:,1:3])
y_val = np.reshape(np.array(val_set[:,0]),[-1,1])
X_val = np.array(val_set[:,1:3])

In [5]:
l_SVM=SVM(Kernel.linear(),0.5)
a=l_SVM.get_alphas(X_train,y_train)
l_SVM.get_w_b(X_train,y_train,a)
yl_pre = l_SVM.predictor(X_val)
acc_l= l_SVM.accuracy(y_val,yl_pre)
print('accuracy is:',acc_l)

accuracy is: 0.5648148148148148


In [6]:
nl_SVM=SVM(Kernel.gaussian(3),0.5)
a_n=nl_SVM.get_alphas(X_train,y_train)
nl_SVM.get_w_b(X_train,y_train,a)
yn_pre = nl_SVM.predictor(X_val)
acc_n= nl_SVM.accuracy(y_val,yn_pre)
print('accuracy is:',acc_n)

accuracy is: 0.8148148148148148
